In [3]:
# this notebook is for sweepign the smush experiment parameters
# I want to see primarily how timestep and U parameter range affect the results

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

import sys
sys.path.append("../../../")

import numpy as np

%matplotlib widget

from src.utils.visualize import unitary_2dlist_weyl, coordinate_2dlist_weyl
from qiskit.quantum_info import Operator
from weylchamber import c1c2c3
import h5py
#from tqdm.notebook import tqdm as tqdm
from tqdm import tqdm

filename = "/home/evm9/decomposition_EM/data/smush_unitary_sweep.h5"

from src.basisv2 import CircuitTemplateV2
from src.utils.custom_gates import CirculatorSNAILGate
from src.cost_function import BasicCostInverse, BasicCost, BasicReducedCost
from src.optimizer import TemplateOptimizer
import h5py

In [5]:
t = 1
duration_1q = .05
gc = np.pi/2
gg = 0*np.pi/4

# Second, we twirl 1Q gates and phase terms given the same gc,gg from (1)

In [6]:
from src.utils.custom_gates import ConversionGainSmushGate

# NOTE first variable is tracking an offset (basically set it to 2 if counting the phase variables)
p_expand = [2, round(t/duration_1q), round(t/duration_1q)]
# XXX turn p_expand into indices is tricky

pp2 =lambda *vargs: ConversionGainSmushGate(vargs[0], vargs[1], gc, gg, vargs[2:2+round(t/duration_1q)], vargs[2+round(t/duration_1q):], t_el=t)
#pp2 =lambda *vargs: ConversionGainSmushGate(0, 0, np.pi/2, 0, vargs[:round(t/du}ration_1q)], vargs[round(t/duration_1q):], t_el=t)

# circuittemplate builds # of parameters by checking the number of parameters in the lambda function to build the gate
# because the parametes for gx and gy are vectors they only get counted once and it messes up
# we can add an extra parameter called param_vec_expand
# we need to use this to tell it to expand the number of parameters we should include
# however, this will get really messy because we don't know which parameters are the vectors or not
# be careful, this is going to be a mess :(
basis = CircuitTemplateV2(n_qubits=2, base_gates=[pp2], no_exterior_1q=1, vz_only=0, param_vec_expand = p_expand)
basis.build(1)
#adding constraint of 1Q params need to be positive valued
#this could be optional let's compare what happens if include it or not
for el in basis.circuit.parameters:
    s_el = str(el)
    if 'Q' in s_el:
        basis.add_bound(s_el, 2*np.pi, -2*np.pi)
#basis.add_bound("Q0", 4*np.pi, 0)
basis.circuit.draw()

»
q_0: »
     »
q_1: »
     »
«     ┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                    ├
«     │  2qsmushgate(Q0,Q1,π/2,0,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40,Q41,1) │
«q_1: ┤1                                                                                                                                                                                    ├
«     └─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [7]:
from src.utils.visualize import unitary_2dlist_weyl
from qiskit.quantum_info import Operator
from tqdm import tqdm
unitary_list = []

for i in tqdm(range(3000)):
    qc_init = basis.assign_Xk(basis.parameter_guess())
    unitary_list.append(Operator(qc_init).data)

unitary_2dlist_weyl(unitary_list)

 41%|████      | 1236/3000 [04:30<06:25,  4.57it/s]


KeyboardInterrupt: 